In [280]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.impute import SimpleImputer
import requests, zipfile 
import io
import os
import scipy.stats as stats
from scipy.fft import fft, fftfreq
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from catboost import Pool, cv
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

In [138]:
import warnings
warnings.filterwarnings('ignore')

In [222]:
import os
X = None
y = None
for file in os.scandir("./bassband_data_new"):
    if 'X' in file.name:
        if X is None:
            X = np.load(file.path)
        else:
            X = np.concatenate((X, np.load(file.path)), axis=0)
    else:
        if y is None:
            y = np.load(file.path)
        else:
            y = np.concatenate((y, np.load(file.path)), axis=0)

In [223]:
X.shape

(38082, 6, 3000)

In [224]:
data = X
data = np.transpose(data, (0,2,1))
data = data.reshape(-1, 6)
df = pd.DataFrame(data)
df.columns = ["EEG Fpz-Cz", "EEG Pz-Oz", "EOG horizontal", "Resp oro-nasal", "EMG submental", "Temp rectal"]
df_10000 = df

In [225]:
FEATURES = ['MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f','Centroid_f']

def features_extraction(df): 
    Max_f=[];Sum_f=[];Mean_f=[];Var_f=[];Peak_f=[];Skew_f=[];Kurtosis_f=[];Centroid_f=[]
    X = df.values
    ft = fft(X)
    S = np.abs(ft**2)/len(df)
    Max_f.append(np.max(S))
    Sum_f.append(np.sum(S))
    Mean_f.append(np.mean(S))
    Var_f.append(np.var(S))
    Peak_f.append(np.max(np.abs(S)))
    Skew_f.append(stats.skew(X))
    Kurtosis_f.append(stats.kurtosis(X))
    freqs = np.fft.fftfreq(len(S))
    spectral_centroid = np.sum(freqs * S) / np.sum(S)
    Centroid_f.append(spectral_centroid)
    features = pd.Series([Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f, Centroid_f], index=FEATURES)
    return features

In [226]:
df_10000

,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal
0,1.107968e-07,5.057362e-07,0.000002,0.247168,0.000003,36.883871
1,7.319898e-07,2.002755e-06,0.000007,0.203765,0.000003,36.883942
2,2.192530e-06,2.192267e-06,0.000013,0.160211,0.000003,36.884016
3,3.998810e-06,-1.187149e-06,0.000011,0.116522,0.000003,36.884093
4,5.091005e-06,-4.187088e-06,0.000007,0.072719,0.000003,36.884173
...,...,...,...,...,...,...
114245995,-1.822526e-06,-1.864197e-07,0.000002,1.236491,0.000004,36.095432
114245996,-1.846713e-06,-4.603951e-07,-0.000008,1.165844,0.000004,36.095688
114245997,1.303461e-06,-1.328989e-06,-0.000007,1.094757,0.000004,36.095946
114245998,3.462724e-06,-1.220276e-06,0.000004,1.023278,0.000004,36.096205


In [ ]:
df_features = pd.DataFrame()

for i in range(0, len(df_10000), 3000):
    segment = df_10000.iloc[i:i+3000]
    features_dict = {}
    for column in segment.columns:  
        features = features_extraction(segment[column])
        for feature_name, feature_value in zip(FEATURES, features):
            features_dict[f'{column}_{feature_name}'] = feature_value
    df_features = df_features.append(features_dict, ignore_index=True)


In [228]:
df_features = df_features.applymap(lambda x: x[0])

In [229]:
df_features

,EEG Fpz-Cz_MAX_f,EEG Fpz-Cz_SUM_f,EEG Fpz-Cz_MEAN_f,EEG Fpz-Cz_VAR_f,EEG Fpz-Cz_PEAK_f,EEG Fpz-Cz_SKEW_f,EEG Fpz-Cz_KURTOSIS_f,EEG Fpz-Cz_Centroid_f,EEG Pz-Oz_MAX_f,EEG Pz-Oz_SUM_f,...,EMG submental_KURTOSIS_f,EMG submental_Centroid_f,Temp rectal_MAX_f,Temp rectal_SUM_f,Temp rectal_MEAN_f,Temp rectal_VAR_f,Temp rectal_PEAK_f,Temp rectal_SKEW_f,Temp rectal_KURTOSIS_f,Temp rectal_Centroid_f
0,3.950157e-09,2.161870e-07,7.206233e-11,6.102794e-20,3.950157e-09,0.029557,1.359038,-3.219199e-10,2.577059e-09,1.537320e-07,...,-0.735407,-9.648715e-23,4.080467e+06,4.080467e+06,1360.155784,5.548219e+09,4.080467e+06,-0.210917,-0.425978,0.000000e+00
1,4.141687e-09,2.581121e-07,8.603737e-11,8.373712e-20,4.141687e-09,0.462991,2.657272,-7.758554e-08,2.022275e-09,8.583081e-08,...,-0.071915,0.000000e+00,4.079959e+06,4.079960e+06,1359.986575,5.546839e+09,4.079959e+06,-0.429482,-0.160684,0.000000e+00
2,4.738611e-09,3.028466e-07,1.009489e-10,1.534501e-19,4.738611e-09,0.386487,2.669680,-2.393443e-09,1.774964e-09,7.994967e-08,...,0.039267,4.270698e-22,4.080514e+06,4.080515e+06,1360.171700,5.548349e+09,4.080514e+06,-0.134407,0.192037,5.314046e-26
3,1.208003e-08,7.731009e-07,2.577003e-10,1.186406e-18,1.208003e-08,0.762955,3.866776,-7.550086e-09,4.666277e-09,1.620242e-07,...,-0.040469,0.000000e+00,4.080364e+06,4.080365e+06,1360.121503,5.547939e+09,4.080364e+06,-0.083593,-0.499623,0.000000e+00
4,2.384008e-08,7.799471e-07,2.599824e-10,1.758521e-18,2.384008e-08,0.382122,2.482101,-1.272094e-09,7.760320e-09,2.305378e-07,...,-0.761223,0.000000e+00,4.078587e+06,4.078588e+06,1359.529306,5.543109e+09,4.078587e+06,-0.054722,-0.248050,2.658278e-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38077,1.042929e-08,3.132917e-07,1.044306e-10,2.432565e-19,1.042929e-08,0.259250,3.833902,-1.799656e-09,3.541534e-09,2.003872e-07,...,-0.664039,4.013885e-23,3.899523e+06,3.899524e+06,1299.841267,5.067071e+09,3.899523e+06,0.468255,0.388341,0.000000e+00
38078,1.841651e-08,3.691661e-07,1.230554e-10,7.745706e-19,1.841651e-08,0.349751,4.703954,-6.468078e-09,9.074346e-09,2.404749e-07,...,15.621775,0.000000e+00,3.900467e+06,3.900468e+06,1300.156023,5.069525e+09,3.900467e+06,0.142852,-0.836033,0.000000e+00
38079,1.633123e-08,3.313978e-07,1.104659e-10,6.914294e-19,1.633123e-08,-0.264926,2.527524,-4.364468e-09,4.942907e-08,6.811848e-07,...,11.416638,0.000000e+00,3.903646e+06,3.903647e+06,1301.215631,5.077792e+09,3.903646e+06,0.497989,0.059918,-2.777408e-26
38080,7.517330e-09,1.762954e-07,5.876513e-11,1.295067e-19,7.517330e-09,0.155390,2.425546,-9.966176e-09,2.849900e-09,8.612072e-08,...,-0.510422,0.000000e+00,3.906732e+06,3.906733e+06,1302.244292,5.085823e+09,3.906732e+06,-0.084935,-0.619380,0.000000e+00


In [230]:
df_features['label'] = y

In [231]:
df_features.to_csv("features_new.csv")

In [235]:
df_copy = pd.read_csv('features_new.csv')

In [309]:
# Evaluation
X_eval_a = np.load('eval_a_X.npy')
X_eval_b = np.load('eval_b_X.npy')

In [310]:
X_eval_a = np.transpose(X_eval_a, (0,2,1))
X_eval_a = X_eval_a.reshape(-1, 6)
X_eval_b = np.transpose(X_eval_b, (0,2,1))
X_eval_b = X_eval_b.reshape(-1, 6)

In [311]:
X_eval_a.shape

(8370000, 6)

In [340]:
df_eval_a = pd.DataFrame(X_eval_a)
df_eval_a.columns = ["EEG Fpz-Cz", "EEG Pz-Oz", "EOG horizontal", "Resp oro-nasal", "EMG submental", "Temp rectal"]
df_eval_b = pd.DataFrame(X_eval_b)
df_eval_b.columns = ["EEG Fpz-Cz", "EEG Pz-Oz", "EOG horizontal", "Resp oro-nasal", "EMG submental", "Temp rectal"]

In [ ]:
df_features_a = pd.DataFrame()
for i in range(0, len(df_eval_a), 3000):
    print(i)
    segment = df_eval_a.iloc[i:i+3000]
    features_dict = {}
    for column in segment.columns:  
        features = features_extraction(segment[column])
        for feature_name, feature_value in zip(FEATURES, features):
            features_dict[f'{column}_{feature_name}'] = feature_value
    df_features_a = df_features_a.append(features_dict, ignore_index=True)

In [ ]:
df_features_b = pd.DataFrame()
for i in range(0, len(df_eval_b), 3000):
    print(i)
    segment = df_eval_b.iloc[i:i+3000]
    features_dict = {}
    for column in segment.columns:  
        features = features_extraction(segment[column])
        for feature_name, feature_value in zip(FEATURES, features):
            features_dict[f'{column}_{feature_name}'] = feature_value
    df_features_b = df_features_b.append(features_dict, ignore_index=True)

In [344]:
df_features_a = df_features_a.applymap(lambda x: x[0])
df_features_b = df_features_b.applymap(lambda x: x[0])

In [345]:
scaler = StandardScaler()
df_features_a = scaler.fit_transform(df_features_a)

scaler = StandardScaler()
df_features_b = scaler.fit_transform(df_features_b)

In [346]:
clf = CatBoostClassifier()
clf.load_model('catboost_model.cbm')
pred_a = clf.predict(df_features_a)

In [347]:
clf = CatBoostClassifier()
clf.load_model('catboost_model.cbm')
pred_b = clf.predict(df_features_b)

In [ ]:
pred_a[0:30]

In [ ]:
pred_b[0:30]

In [ ]:
unique, counts = np.unique(pred_a, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
unique, counts = np.unique(pred_b, return_counts=True)

print(np.asarray((unique, counts)).T)

In [352]:
pred_a = pred_a.reshape(-1)
pred_b = pred_b.reshape(-1)

In [353]:
np.save('a_pred.npy', pred_a)
np.save('b_pred.npy', pred_b)